In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Data_mining import data_mining

In [4]:
df=data_mining("ETH-USDT", "5m", "2021/01/01", "2023/07/31")

In [5]:
df

,open,close,high,low,volume
time,,,,,
2021-01-01 00:00:00,738.52,739.09,739.09,737.71,9102.55
2021-01-01 00:05:00,739.09,739.65,740.49,738.83,9580.63
2021-01-01 00:10:00,739.65,739.18,740.11,738.98,8915.66
2021-01-01 00:15:00,739.18,739.07,739.87,738.75,8787.57
2021-01-01 00:20:00,739.10,741.32,741.51,738.94,9586.71
...,...,...,...,...,...
2023-07-29 23:35:00,1878.61,1880.37,1881.14,1878.58,5839.04
2023-07-29 23:40:00,1880.37,1881.17,1881.71,1880.17,1885.75
2023-07-29 23:45:00,1881.18,1880.35,1882.10,1880.29,2697.89


In [7]:
df.to_csv("ETH-USDT_5m.csv")